In [9]:
import tensorflow as tf

<h3 style="color:purple">Create tf dataset from a list</h3>

In [10]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<h3 style="color:purple">Iterate through tf dataset</h3>

In [11]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


<h3 style="color:purple">Iterate through elements as numpy elements</h3>

In [12]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


<h3 style="color:purple">Iterate through first n elements in tf dataset</h3>

In [13]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<h3 style="color:purple">Filter sales numbers that are &lt; 0</h3>

In [14]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


<h3 style="color:purple">Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1-&gt;72 conversation rate</h3>

In [15]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


<h3 style="color:purple">Shuffe</h3>

In [16]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2232
2448


<h3 style="color:purple">Batching</h3>

In [17]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 1584]
[2232 2448]
[2304 2232]


<h3 style="color:purple">Perform all of the above operations in one shot</h3>

In [18]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 2232]
[2304 1512]
[2448 2232]


In [50]:
images_ds = tf.data.Dataset.list_files('training_set/training_set/*/*', shuffle=False)

In [51]:
image_count = len(images_ds)
image_count


8007

In [52]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [56]:
for file in images_ds.take(3):
    print(file.numpy())

b'training_set\\training_set\\cats\\cat.110.jpg'
b'training_set\\training_set\\cats\\cat.1169.jpg'
b'training_set\\training_set\\cats\\_DS_Store'


In [57]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'training_set\\training_set\\cats\\cat.1029.jpg'
b'training_set\\training_set\\cats\\cat.1045.jpg'
b'training_set\\training_set\\cats\\cat.113.jpg'


In [58]:
class_names = ["cat","dog"]


In [59]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [60]:
len(train_ds)

6405

In [61]:
len(test_ds)

1602

In [65]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [67]:
get_label("training_set\\training_set\\cats\\cat.1169.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'cats'>

In [68]:
s="training_set\\training_set\\cats\\cat.1169.jpg"
s.split("\\")[-2]

'cats'

In [69]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [70]:
img, label = process_image("training_set\\training_set\\cats\\cat.1169.jpg")
img.numpy()[:2]

array([[[155.9375  , 156.9375  , 138.9375  ],
        [154.      , 155.      , 137.      ],
        [154.3164  , 155.3164  , 137.3164  ],
        [153.5625  , 154.5625  , 136.5625  ],
        [154.5625  , 155.5625  , 137.5625  ],
        [155.      , 156.      , 138.      ],
        [153.69214 , 154.69214 , 136.69214 ],
        [152.6211  , 153.6211  , 135.6211  ],
        [153.96118 , 154.96118 , 136.96118 ],
        [152.30786 , 153.30786 , 135.30786 ],
        [153.      , 154.      , 136.      ],
        [153.      , 154.      , 136.      ],
        [153.      , 154.      , 136.      ],
        [151.      , 152.      , 134.      ],
        [151.      , 152.      , 134.      ],
        [151.      , 152.      , 134.      ],
        [149.9375  , 150.9375  , 132.9375  ],
        [148.30786 , 149.30786 , 131.30786 ],
        [147.11841 , 148.11841 , 130.11841 ],
        [149.      , 150.      , 132.      ],
        [150.5625  , 151.5625  , 133.5625  ],
        [150.      , 151.      , 1

In [71]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [72]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 45.980637  54.980637  53.980637]
  [ 49.347656  58.347656  57.347656]
  [ 48.2899    57.2899    56.2899  ]
  ...
  [122.2726    92.2726   102.2726  ]
  [119.38281   89.38281  101.38281 ]
  [110.31641   83.31641   90.34799 ]]

 [[ 50.34375   56.34375   56.34375 ]
  [ 52.        58.        58.      ]
  [ 55.920486  61.920486  61.920486]
  ...
  [121.24609   90.35156   96.5625  ]
  [116.38432   85.48979   91.70073 ]
  [108.395035  82.34766   84.45703 ]]

 [[ 53.82422   57.82422   58.82422 ]
  [ 53.        57.        58.      ]
  [ 54.89453   58.89453   59.89453 ]
  ...
  [119.421875  89.37862   91.50839 ]
  [112.80998   82.80998   82.0862  ]
  [101.875     73.13449   70.20871 ]]

 ...

 [[ 22.596817  21.596817  16.596817]
  [ 24.641754  23.641754  18.641754]
  [ 26.246094  25.246094  20.246094]
  ...
  [ 89.842896  99.64006  103.654816]
  [110.98341  117.98341  126.20746 ]
  [116.96481  122.88585  136.7338  ]]

 [[ 19.145035  18.145035  13.145035]
  [ 24.006302  23.006

In [73]:
def scale(image, label):
    return image/255, label

In [74]:
train_ds = train_ds.map(scale)

In [77]:
for image, label in train_ds.take(100):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.1586397  0.22945772 0.26798406]
****Label:  b'cats'
****Image:  [0.92359066 0.856924   0.73927695]
****Label:  b'cats'
****Image:  [0.10630996 0.17637604 0.15269476]
****Label:  b'cats'
****Image:  [0.6902918  0.7630247  0.69153357]
****Label:  b'cats'
****Image:  [0.99215686 0.99215686 0.99215686]
****Label:  b'cats'
****Image:  [0.04313726 0.04705882 0.02352941]
****Label:  b'cats'
****Image:  [0.47293702 0.4141135  0.4023488 ]
****Label:  b'cats'
****Image:  [0.14933424 0.1532558  0.13364796]
****Label:  b'cats'
****Image:  [0.10955637 0.13700736 0.16485661]
****Label:  b'cats'
****Image:  [0.17290135 0.16897978 0.08662684]
****Label:  b'cats'
****Image:  [0.01241754 0.00854193 0.32595628]
****Label:  b'cats'
****Image:  [0.5958793  0.54489887 0.4821538 ]
****Label:  b'cats'
****Image:  [0.4303682  0.29311332 0.16762312]
****Label:  b'cats'
****Image:  [0.1851103  0.18750419 0.21169506]
****Label:  b'cats'
****Image:  [0.67546767 0.6415829  0.6218075 ]
****Label:  b'c